In [2]:
import os

In [3]:
print("All the Images")
for i in os.listdir("../Storage"):
    print("Name: {}\tEmail:{}".format(i[0:i.find("_")], i[i.find("_")+1:]))
    ctName = 0
    for j in os.listdir("../Storage/" + i):
        print("\t",j)
        #os.rename("../Storage/" + i + "/" + j,"Img_"+ str(0) +".jpg")
        #Instead of renaming create new folder within it and duplicate Images there

All the Images
Name: robert	Email:robertDowney202@gmail.com
	 download (2).jpg
	 download (3).jpg
	 download (4).jpg
	 download (5).jpg
	 download (6).jpg
	 download.jpg
	 images (1).jpg
	 images (2).jpg
	 images.jpg
Name: tom	Email:Tomholand1@gmail.com
	 download (1).jpg
	 download (2).jpg
	 download.jpg
	 image.jpg
	 images (1).jpg
	 images (2).jpg
	 images (3).jpg
	 images.jpg
	 tom-holland-photo-jason-kempin-getty-images-801510482-profile.jpg
	 Tom_Holland_by_Gage_Skidmore.jpg


In [4]:
from PIL import Image
imgPath = "../Storage/tom_Tomholand1@gmail.com/image.jpg"
im = Image.open(imgPath)

In [5]:
#im

In [26]:
from google.cloud import storage
def list_blobs(bucket_name):
    storage_client = storage.Client.from_service_account_json(key)
    blobs = storage_client.list_blobs(bucket_name)
    for blob in blobs:
        print(blob.name)
def list_buckets():
    storage_client = storage.Client.from_service_account_json(key)
    buckets = storage_client.list_buckets()
    for bucket in buckets:
        print(bucket.name)

In [27]:
#KEYDownloadedName = "key.json"
#bucket = storage_client.bucket("loginly_storage")
#blob = bucket.get_blob("GCPKEY/loginly.json")
#blob.download_to_filename(KEYDownloadedName)

In [ ]:
# This is the function to take in the 10 image names and the user's email and user's 
# name and crops the image and upload to proper folder in GCP 
#-----
# INPUT:   IMG1, IMG2, IMG3 ... IMG10, NAME, EMAIL
#--------------
key = "../GCPKEY/loginly.json"
harascade = '../Models/haarcascades/haarcascade_frontalface_alt2.xml'
# key.json                              file on function      
# haarcascade_frontalface_alt2.xml      file on function 
#--------------
#fileName0 = 
name = "robert"
email = "robertDowney202@gmail.com"
folder = name + "_" + email
fileName1 = "download (2).jpg"
fileName2 = "download (3).jpg"
fileName3 = "download (4).jpg"
fileName4 = "download (5).jpg"
fileName5 = "download (6).jpg"
fileName6 = "download.jpg"
fileName7 = "images (1).jpg"
fileName8 = "images (2).jpg"
fileName9 = "images.jpg"
lstOfFiles = [fileName1,fileName2,fileName3,fileName4,fileName5,fileName6,fileName7,fileName8,fileName9]
#-------------
os.chdir("/Users/kunal/Documents/AAPersonalAIPROJECT/LOGINLY/Code") # Comment later
storage_client = storage.Client.from_service_account_json(key) #"key.json"
bucket_name = 'loginly_storage'
bucket = storage_client.bucket(bucket_name)
ctFileName = 1
for i in lstOfFiles:
    blob = bucket.blob("Storage/"+folder+"/" + i)
    blob.download_to_filename("Img_org_" + str(ctFileName) + ".jpg")
    ctFileName+=1
#--------------
downloadedFilesName = []
for i in range(len(lstOfFiles)):
    downloadedFilesName.append("Img_org_" + str(i+1) + ".jpg")
#-------------
os.mkdir(folder + "_CLEAN")
ctFileNameNew = 1
newFilesName = []
for i in downloadedFilesName:
    img = cv2.imread(i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(harascade)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    #1.1: scale factor, it specifies how much the image size is reduced with each scale. It improves detection.
    #4: MinNeighbours, specifies how many neighbors each candidate rectangle should have to retain.
    if len(faces) > 0:
        x,y,w,h = faces[0]
    else:
        x,y = 0,0
        w,h = img.shape[0:2]
    crop_img = img[y:y+h, x:x+w]
    currentDir = os.getcwd()
    os.chdir(folder + "_CLEAN")
    cv2.imwrite("Img_NEW_" + str(ctFileNameNew) + ".jpg", crop_img)
    os.chdir(currentDir)
    newFilesName.append("Img_NEW_" + str(ctFileNameNew) + ".jpg")
    ctFileNameNew+=1
#-------------- 
#upload new files
os.chdir(folder + "_CLEAN")
for i in newFilesName:
    blob = bucket.blob("CleanData/" +folder+"/"+ i)
    blob.upload_from_filename(i)

In [66]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os